In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from utils import data, plot, evaluate
import models
from models import GRU, LSTM

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Load Data

In [ ]:
data_dir = 'data/CMAPSS'
subset = 1
train, test, header, n_meta, n_sensors, n_settings = data.load_cmapss_subset(
    data_dir, subset)

params = {
    'drop_sensors': [1, 5, 6, 10, 16, 18, 19],
    'include_settings': None,
    'smooth': 0,
    'lag': 20,
    'clip': 125,
    'cv_folds': None,
    'return_cols': True,
    'reshape_2d': True
}
X, y, _, cols = data.build_dataset(train, **params)
X_test, y_test, _, cols = data.build_dataset(test, **params, test=True)

y = np.expand_dims(y, axis=1)
y_test = np.expand_dims(y_test, axis=1)
print(X.shape, y.shape)

batch_size = 2048

train_data = TensorDataset(
    torch.from_numpy(X),
    torch.from_numpy(y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

test_data = TensorDataset(
    torch.from_numpy(X_test),
    torch.from_numpy(y_test))
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=False)

In [ ]:
model_params = {
    'batch_size': batch_size,
    'learning_rate': 0.001,
    'epochs': 3,
    'model_type': 'GRU',
    'criterion': nn.MSELoss(),
}

print(f'{model_params["model_type"]} Model\n------------------')
model, history = models.train(device, train_loader, **model_params)
print('---')
y_pred = models.predict(device, gru_model, X_test, y_test)
metrics = evaluate.metrics(y_test, y_pred)
print('Test Set - MAPE={:.3f} - MAE={:.3f} - MSE={:.3f} - RMSE={:.3f} - R2={:.3f}'.format(*metrics))

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].plot(history['losses'])
axes[1].plot(y_test, y_pred, '.')
axes[1].set_ylabel('Predictions')
axes[1].set_xlabel('Ground Truth')
axes[1].set_title('Test Set Predictions')